In [1]:
import pandas as pd
import gzip
import sweetviz as sv
import os
import glob

/Users/camillecu/Downloads/KUL/MDA/project/waffle/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Camille -- PIT

In [12]:

pit_loc = "/Users/camillecu/Downloads/KUL/MDA/project/dataset/pit_locations.parquet.gzip"
pit_loc = pd.read_parquet(pit_loc)


In [9]:
pit_loc.head()

,unit,campus,province,region,unit_id,ambucode,ambusitecode,link
0,"100001: PIT Rumst, AZ Rivierland",Ziekenhuis/Hôpital: 104 - AZ RIVIERENLAND --- ...,Antwerpen/Anvers,Vlaanderen/Flandre,PARUMS01A,191,19101,None
1,"102001: PIT Deinze, St Vincentius",Ziekenhuis/Hôpital: 134 - AZ SINT-VINCENTIUS -...,Oost-Vlaanderen/Flandre orientale,Vlaanderen/Flandre,PODEIN01A,413,41301,None
2,102101: PIT Stuivenberg,Ziekenhuis/Hôpital: 009 - ZIEKENHUISNETWERK AN...,Antwerpen/Anvers,Vlaanderen/Flandre,PAANTW01A,117,11704,None
3,"110001: PIT Lier, Heilig Hart",Ziekenhuis/Hôpital: 097 - HEILIG HART ZIEKENHU...,Antwerpen/Anvers,Vlaanderen/Flandre,PALIER01A,171,17101,None
4,113001: PIT HEILIG HART ZIEKENHUIS,Ziekenhuis/Hôpital: 102 - HEILIG HART ZIEKENHU...,Antwerpen/Anvers,Vlaanderen/Flandre,PAMOL_01A,190,19001,None


In [4]:
num_rows = pit_loc.shape[0]
print("Number of rows in pit dataset:", num_rows)

Number of rows in pit dataset: 24


In [13]:
# Specify the file path
file_path = "/Users/camillecu/Downloads/KUL/MDA/project/dataset/variables_ambureg.xlsx"

# Read the xlsx file into a DataFrame
ambureg = pd.read_excel(file_path, header=0)

# Display the DataFrame
ambureg.head()

,old ambulance code,ambulance code,site number,y,x,address,locality,ZIP,Province
0,167,35,3511,51.119490,4.667580,Pastorijstraat 25,BERLAAR,2590.0,Antwerpen
1,255,35,3522,50.743246,4.241075,Brusselsesteenweg 183,HALLE,1500.0,Vlaams-Brabant
2,315,35,3531,51.002498,3.322729,Bruggestraat 84,TIELT,8700.0,West-Vlaanderen
3,735,35,3571,51.248576,5.525886,Steenstraat 5,HAMONT-ACHEL,3930.0,Limburg
4,225,36,3621,50.660384,4.567357,Rue De Franquenies 10,OTTIGNIES,1341.0,Brabant Wallon


In [14]:
# merge x, y, address from ambureg to pit_loc where ambusitecode is the same as site number in ambureg
# only keep the columns x, y, address from ambureg
pit_loc = pd.merge(pit_loc, ambureg, left_on='ambusitecode', right_on='site number', how='left')


In [8]:
pit_loc.columns

Index(['unit', 'campus', 'province', 'region', 'unit_id', 'ambucode',
       'ambusitecode', 'link', 'old ambulance code', 'ambulance code',
       'site number', 'y', 'x', 'address', 'locality', 'ZIP', 'Province'],
      dtype='object')

In [15]:
# only keep the columns that are needed: unit, campus, province, region, unit_id, ambucode, ambusitecode, x, y, address
pit_loc = pit_loc[['unit', 'campus', 'province', 'region', 'unit_id', 'ambucode', 'ambusitecode', 'x', 'y', 'address','ZIP']]

In [16]:
# Change column names from 'x' to 'latitude' and 'y' to 'longitude'
pit_loc = pit_loc.rename(columns={'x': 'latitude', 'y': 'longitude'})

In [17]:
pit_loc




,unit,campus,province,region,unit_id,ambucode,ambusitecode,latitude,longitude,address,ZIP
0,"100001: PIT Rumst, AZ Rivierland",Ziekenhuis/Hôpital: 104 - AZ RIVIERENLAND --- ...,Antwerpen/Anvers,Vlaanderen/Flandre,PARUMS01A,191,19101,4.370861,51.105468,S Herenbaan 172,2840.0
1,"102001: PIT Deinze, St Vincentius",Ziekenhuis/Hôpital: 134 - AZ SINT-VINCENTIUS -...,Oost-Vlaanderen/Flandre orientale,Vlaanderen/Flandre,PODEIN01A,413,41301,3.527715,50.986620,Meulenstraat,9800.0
2,102101: PIT Stuivenberg,Ziekenhuis/Hôpital: 009 - ZIEKENHUISNETWERK AN...,Antwerpen/Anvers,Vlaanderen/Flandre,PAANTW01A,117,11704,NaN,NaN,NaN,NaN
3,"110001: PIT Lier, Heilig Hart",Ziekenhuis/Hôpital: 097 - HEILIG HART ZIEKENHU...,Antwerpen/Anvers,Vlaanderen/Flandre,PALIER01A,171,17101,NaN,NaN,NaN,NaN
4,113001: PIT HEILIG HART ZIEKENHUIS,Ziekenhuis/Hôpital: 102 - HEILIG HART ZIEKENHU...,Antwerpen/Anvers,Vlaanderen/Flandre,PAMOL_01A,190,19001,5.114858,51.186947,Gasthuisstraat 1,2400.0
5,200001: PIT LOKEREN,Ziekenhuis/Hôpital: 595 - VITAZ --- Campus/Sit...,Oost-Vlaanderen/Flandre orientale,Vlaanderen/Flandre,POLOKE01A,440,44001,NaN,NaN,NaN,NaN
6,"210001: PIT ASSE, OLV",Ziekenhuis/Hôpital: 126 - ONZE-LIEVE-VROUW ZIE...,Vlaams Brabant/Brabant flamand,Vlaanderen/Flandre,PVASSE01A,480,48001,NaN,NaN,Bloklaan 5,1730.0
7,301001: PIT ST Jan (Associatie St Lucas),Ziekenhuis/Hôpital: 049 - AZ SINT-JAN BRUGGE -...,West-Vlaanderen/Flandre occidentale,Vlaanderen/Flandre,PWBRUG01A,391,39102,3.248934,51.184830,Sint-Lucaslaan 29,8310.0
8,301001: PIT St Lucas Brugge (associatie St Jan),Ziekenhuis/Hôpital: 140 - AZ SINT-LUCAS --- Ca...,West-Vlaanderen/Flandre occidentale,Vlaanderen/Flandre,PWBRUG02A,391,39101,3.194468,51.220337,Ruddershove 10\n,8000.0
9,303001: PIT Blankenberge,Ziekenhuis/Hôpital: 392 - AZ ZENO --- Campus/S...,West-Vlaanderen/Flandre occidentale,Vlaanderen/Flandre,PWBLAN01A,374,37401,NaN,NaN,NaN,NaN


In [ ]:
# print the row that contains 